# Getting Data from OECD


In [1]:
import pandas_datareader.data as web
import pandas as pd

C:\Users\nicol\Anaconda3\lib\site-packages\pandas_datareader\compat\__init__.py:9: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


### Gross Domestic Expenditure on R&D (GERD) as percentage of GDP

In [2]:
GERD_reader = web.DataReader('MSTI_PUB', 'oecd', start=1992, end=2017)

In [3]:
GERDdf = GERD_reader['GERD as a percentage of GDP']

# Need to change from separate columns for each country to repeated rows
# and a column for the percent value
GERDdf = GERDdf.stack().to_frame()
GERDdf.reset_index(inplace=True)
GERDdf.sort_values(['Country', 'Year'], inplace=True)
GERDdf.reset_index(drop=True, inplace=True)
GERDdf.rename({0:'Percent_GDP_on_RD'}, axis=1, inplace=True)

GERDdf.head()

,Year,Country,Percent_GDP_on_RD
0,1992-01-01,Argentina,0.359716
1,1993-01-01,Argentina,0.403760
2,1994-01-01,Argentina,0.452155
3,1995-01-01,Argentina,0.470545
4,1996-01-01,Argentina,0.564050


### Percent of Population with Health Insurance (Public or Private)

In [4]:
HI_reader = web.DataReader('HEALTH_PROT', 'oecd', start=1992, end=2017)

In [5]:
HIdf = HI_reader['Total public and primary private health insurance']['% of total population covered']

# Need to change from separate columns for each country to repeated rows
# and a column for the percent value
HIdf = HIdf.stack().to_frame()
HIdf.reset_index(inplace=True)
HIdf.sort_values(['Country', 'Year'], inplace=True)
HIdf.reset_index(drop=True, inplace=True)
HIdf.rename({0:'Percent_Pop_HI'}, axis=1, inplace=True)

HIdf.head()

,Year,Country,Percent_Pop_HI
0,1992-01-01,Australia,100.0
1,1993-01-01,Australia,100.0
2,1994-01-01,Australia,100.0
3,1995-01-01,Australia,100.0
4,1996-01-01,Australia,100.0


### Production-based CO2 productivity, GDP per unit of energy-related CO2 emissions

In [6]:
CO2_reader = web.DataReader('GREEN_GROWTH', 'oecd', start=1992, end=2017)

In [7]:
# This one is different because each country had a separate table
# instead of all being a column in one table. Below is the code that
# combines all of the tables into one dataframe.


# Initializing empty dataframe
CO2df = pd.DataFrame(columns=['Year', 'Country', 'GDP_per_unit_CO2'])

# CO2_reader has a multi-index. The country is the first entry of each,
# so the following code pulls out the country names and set() gets rid of duplicates.
countries = set(CO2_reader.columns.get_level_values(0))
countries = sorted(list(countries))

# This for loop gets the particular data we are interested in for 
# each country and then adds that to the master dataframe:
for country in countries:
    Cseries = CO2_reader[country]['Production-based CO2 productivity, GDP per unit of energy-related CO2 emissions']
    Cdf = Cseries.to_frame()
    Cdf.reset_index(inplace=True)
    Cdf.rename({'Production-based CO2 productivity, GDP per unit of energy-related CO2 emissions':'GDP_per_unit_CO2'},
                 axis=1, inplace=True) 
    Cdf.insert(1, 'Country', country)
    CO2df = pd.concat([CO2df, Cdf], axis=0)


# Countries (and some groups) that are missing all or some of the years:
# ASEAN, Afghanistan, American Samoa, Andorra, Anguilla, Antartica, Antigua and Barbuda,
# Aruba, Bahamas, Barbados, Belize, Bermuda, Bhutan, Burkina Faso, Burundi, Cabo Verde,
# Cambodia (92-94), Central African Republic, Chad, Djibouti, Equatorial Guinea, Eswatini,
# Euro area (19 countries) (92-94), Fiji, Gambia, Guinea, Kiribati, Lao People's Democratic Republic,
# Latin America and Caribbean, Lesotho, Liberia, Liechtenstein, Madagascar, Malawi, Maldives, Mali,
# Martinique, Mauritania, Mayotte, Micronesia, Middle East and North Africa, Monaco, Montenegro (92-2004),
# Montserrat, Nauru, Niger (92-99), Palau, Papua New Guinea, Puerto Rico, Rwanda, Samoa, San Marino,
# Sao Tome and Principe, Seychelles, Sierra Leone, Solomon Islands, Somalia, Suriname (92-99), Tokelau,
# Tonga, Turks and Caicos Islands, Tuvalu, Uganda, Vanuatu
# Removing those:
CO2df.dropna(inplace=True)
CO2df.reset_index(drop=True, inplace=True)


CO2df.head()

,Year,Country,GDP_per_unit_CO2
0,1992-01-01,Albania,4.443426
1,1993-01-01,Albania,5.264840
2,1994-01-01,Albania,5.542105
3,1995-01-01,Albania,6.905429
4,1996-01-01,Albania,7.477210


### Merging Data Together

In [10]:
merge1 = pd.merge(left=GERDdf, right=HIdf, how='outer',
                       left_on=['Country', 'Year'], right_on=['Country', 'Year'])
merged = pd.merge(left=merge1, right=CO2df, how='outer',
                       left_on=['Country', 'Year'], right_on=['Country', 'Year'])
merged.sort_values(['Country', 'Year'], inplace=True)
merged.reset_index(drop=True, inplace=True)

### Fixing Country Names

In [11]:
import pycountry
# Official/Up-to-date Country Names
pycountries = [country.name for country in pycountry.countries]

# Country names that appear in OECD data
mergedcountries = sorted(list(set(merged.iloc[:,1])))

# Making csv for me to see what needs to be changed
Same = sorted(list(set(mergedcountries).intersection(set(pycountries))))
Merged_Only = sorted(list(set(mergedcountries) - set(pycountries)))
Py_Only = sorted(list(set(pycountries) - set(mergedcountries)))
comparisondf = pd.DataFrame({'Same': pd.Series(Same),
                             'Merged_Only': pd.Series(Merged_Only),
                             'Py_Only': pd.Series(Py_Only)})
# comparisondf.to_csv('comp.csv')

In [12]:
# OECD has some groups of countries that I will remove
remove = ['ASEAN', 'BRIICS economies - Brazil, Russia, India, Indonesia, China and South Africa',
         'Eastern Europe, Caucasus and Central Asia', 'Euro area (19 countries)',
         'European Union (15 countries)', 'European Union (28 countries)',
         'G20', 'G7', 'Latin America and Caribbean', 'Middle East and North Africa',
         'OECD - Europe', 'OECD - Total', 'OECD America', 'OECD Asia Oceania', 'World']

merged2 = merged[~merged.Country.isin(remove)]

In [13]:
# Other country names need to be updated to match current official names
update_dic = {'Bolivia':'Bolivia, Plurinational State of',
              "China (People's Republic of)":'China',
              'Chinese Taipei':'Taiwan, Province of China',
              'Czech Republic':'Czechia',
              'Democratic Republic of the Congo':'Congo, The Democratic Republic of the',
              'Iran':'Iran, Islamic Republic of',
              'Korea':'Korea, Republic of',
              'Micronesia':'Micronesia, Federated States of',
              'Moldova':'Moldova, Republic of',
              'Russia':'Russian Federation',
              'Slovak Republic':'Slovakia',
              'Tanzania':'Tanzania, United Republic of',
              'Venezuela':'Venezuela, Bolivarian Republic of'}
merged3 = merged2.replace({"Country": update_dic})

In [15]:
# Checking that it worked
mergedcountries = sorted(list(set(merged3.iloc[:,1])))
Same = sorted(list(set(mergedcountries).intersection(set(pycountries))))
Merged_Only = sorted(list(set(mergedcountries) - set(pycountries)))
Py_Only = sorted(list(set(pycountries) - set(mergedcountries)))
comparisondf2 = pd.DataFrame({'Same': pd.Series(Same),
                             'Merged_Only': pd.Series(Merged_Only, dtype='float64'),
                             'Py_Only': pd.Series(Py_Only)})
# comparisondf2.to_csv('comp2.csv')
# All good, except there are pycountries not in the OECD data